In [1]:
import datetime
start_time = datetime.datetime.now(); print(start_time)

2019-04-26 18:29:21.758000


#Child repartition

### Table of contents:
* [Load Data](#Load-Data)
* [Match individual tax returns](#Match-avec-les-déclarations-fiscales-individuelles)

In [2]:
%matplotlib inline

In [3]:
year = 2013

In [4]:
import matplotlib.pyplot as plt

In [5]:
from __future__ import division
import pandas as pd ; pd.set_option("display.max_columns",200)

In [6]:
 check_for_error = False

In [7]:
store_path = '../Data/hdf/edp_2015_final.h5'
Store = pd.HDFStore(store_path)

In [8]:
def load_fisc_i_by_year(year = 2014):
    df_fisci = pd.read_hdf(store_path, 'FISC_INDIVIDU_{}'.format(year))
    return df_fisci
def load_fisc_r_by_year(year = 2014):
    df_fiscr = pd.read_hdf(store_path, 'FISC_REVENU_{}'.format(year))
    return df_fiscr
def load_fisc_l_by_year(year = 2014):
    df_fiscr = pd.read_hdf(store_path, 'FISC_LOGEMENT_{}'.format(year))
    return df_fiscr

def load_df_fiscrevdet_by_year(year = 2015):
    df_fiscrevdet = pd.read_hdf(store_path, 'FISC_REVDET_{}'.format(year))
    return df_fiscrevdet
    

In [9]:
path = (u"./Pickle/")
import pickle
def select_id_fisc_log(year = None):
    return pickle.load(open(path+"final_select_rev_{}.p".format(year), 'rb'))

In [10]:
select_id_fisc_by_year = {year:select_id_fisc_log(year) for year in range(2010,2015)}

In [11]:
#df = load_fisc_i_by_year(2013)

In [12]:
#df.shape

## Load Data

In [13]:
%%time
df_ref =  load_fisc_i_by_year(2010) #dataframe de reference
df_ref= df_ref[df_ref.ID_FISC_LOG_DIFF.isin(select_id_fisc_by_year[2010])]
print df_ref.shape
for year in range(2011,2015):
    df_temp = load_fisc_i_by_year(year) #dataframe de reference
    df_temp = df_temp[df_temp.ID_FISC_LOG_DIFF.isin(select_id_fisc_by_year[year])]
    print year, df_temp.shape
    print df_temp.AN_FISC.iloc[0]
    df_ref = pd.concat([df_ref,df_temp])

(647161, 28)
2011 (673528, 28)
2012
2012 (687398, 28)
2013
2013 (688806, 28)
2014
2014 (699691, 28)
2015
Wall time: 1min 41s


### Stat répartition des enfants dans les logements fiscaux (equal split ?).

In [14]:
%%time
df_dict = dict()
for year in range(2010,2015):
    df = df_ref[(df_ref.AN_FISC-1) == year]
    print year
    df.ID_FISC_LOG_DIFF.value_counts().index.value_counts().shape
    grpby = df.groupby('ID_FISC_LOG_DIFF')
    print grpby['ID_FISC_FOY_DIFF'].nunique().value_counts()
    #Au cas ou ça ne passe pas (on a pas toujours 2 foyers pour 1 logement), décomenter
    #keep = grpby['ID_FISC_FOY_DIFF'].nunique() == 2
    #to_keep = keep[keep].index
    #df = df[df.ID_FISC_LOG_DIFF.isin(to_keep)]

    
    assert df.ID_FISC_LOG_DIFF.value_counts().index.value_counts().shape[0] == df.ID_FISC_LOG_DIFF.unique().shape[0]
    assert( df.ID_FISC_LOG_DIFF.value_counts().index.value_counts().shape[0]*2 ==
        df.ID_FISC_FOY_DIFF.value_counts().index.value_counts().shape[0])
    df_dict[year] = df


2010
2    165154
dtype: int64
2011
2    171352
dtype: int64
2012
2    174304
dtype: int64
2013
2    173865
dtype: int64
2014
2    175693
dtype: int64
Wall time: 1min 6s


In [15]:
df.groupby('ID_FISC_LOG_DIFF')['ID_FISC_FOY_DIFF'].nunique().value_counts()

2    175693
dtype: int64

On a bien le même nombre de logements, de logements unique, et le double de foyers fiscaux...

In [16]:
%%time
if check_for_error == True:
    for year in range(2010,2015):
        df = df_ref[(df_ref.AN_FISC-1) == year]
        print df.groupby('ID_FISC_LOG_DIFF')['ID_FISC_FOY_DIFF'].nunique().value_counts()
        print df.groupby('ID_FISC_LOG_DIFF')['ID_FISC_FOY_DIFF'].nunique().isin([1,3]).value_counts()

Wall time: 0 ns


Tout les logements ont 2 foyers fiscaux

In [17]:
df_ref['ID_FISC_IND_DIFF'] = df_ref['ID_FISC_FOY_DIFF'].map(str)+ df_ref['ORDREFIP'].map(str) + df_ref[ 'TYPE_FISC'].map(str)

In [18]:
%%time
df_list = list()
for year in range(2010,2015):
    df = df_ref[(df_ref.AN_FISC-1) == year]
    nb_enf_by_foy = pd.DataFrame(df.groupby('ID_FISC_FOY_DIFF')['ID_FISC_IND_DIFF'].nunique()-1)
    nb_enf_by_foy.columns = ['nb_enf_foy']
    df2 = pd.merge(df, nb_enf_by_foy, left_on = 'ID_FISC_FOY_DIFF', right_index=True, how='outer')
    nb_enf_by_log = pd.DataFrame(df2.groupby('ID_FISC_LOG_DIFF')['ID_FISC_IND_DIFF'].nunique()-2)
    nb_enf_by_log.columns = ['nb_enf_log']
    df2 = pd.merge(df2, nb_enf_by_log, left_on = 'ID_FISC_LOG_DIFF', right_index=True, how='outer')
    df_list.append(df2)

Wall time: 3min 27s


In [19]:
%%time
df_ref = pd.concat(df_list)

Wall time: 6.76 s


In [20]:
#Depecated

In [21]:
#df3 = df3[['ID_DIFF','AN_FISC','ID_FISC_FOY_DIFF','ID_FISC_LOG_DIFF','ID_FISC_IND_DIFF','ORDREFIP','TYPE_FISC', 'nb_enf_foy', 'nb_enf_log','SEXE','JNAIS','MNAIS', 'ANAIS','CODNAIS', 'CIDECI', 'ZOXYZD', 'DACOED', 'ZOXYZP', 'DACOEP', 'T_CHARGE', 'CNCOIPV', 'CSDEP', 'CNE1', 'COM_NSCAT', 'COM_SCAT1', 'COM_SCAT2', 'I_FISC_LOGEMENT', 'I_FISC_REVENU','TYPE_DECL',]]

### Répartition des parts des enfants dans les foyers fiscaux

In [22]:
h = df_ref.groupby(['ID_FISC_FOY_DIFF','AN_FISC'])

In [23]:
h2 = h.mean()

In [24]:
h2.pivot_table(columns = 'nb_enf_log' )

nb_enf_log,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,16.0
ANAIS,1966.403926,1983.408151,1986.838609,1988.327053,1991.993878,1992.048618,1995.224612,1996.771281,1998.278267,1998.420403,1999.046369,1999.606001,1999.644061,2002.500000,1996.833333
CNE1,236.143693,226.551898,227.929072,234.251383,238.083623,235.863546,244.551521,241.259494,250.409274,255.124424,229.182609,231.918919,245.120000,117.000000,49.000000
COM_NSCAT,1.000000,1.017370,1.017076,1.035398,1.000000,1.111111,1.100000,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
COM_SCAT1,30.000000,28.756824,27.472785,27.707965,23.071429,24.222222,16.500000,12.000000,14.000000,NaN,NaN,NaN,NaN,NaN,NaN
COM_SCAT2,0.000000,0.390244,0.371134,1.560000,0.000000,2.800000,1.750000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
DACOED,13815166.472464,12960556.295534,12783048.246491,12161773.164504,12354354.121793,12079381.311202,11746345.564787,12168284.327869,11575861.955414,15698673.753623,14490369.540984,8383672.000000,17496554.181818,9032011.000000,27022012.000000
DACOEP,13540868.727273,15210196.344537,15459516.308163,15494306.087811,15735350.038493,15387232.678934,15973458.441327,14324097.546512,11530363.268657,12948878.562500,13048677.000000,NaN,6082005.000000,NaN,NaN
ID_DIFF,2253027.922701,2253043.382049,2204139.146590,2251678.839299,2466477.310040,2518565.039461,2596125.408653,2763219.353561,2761439.745588,2955560.769397,2922645.832661,3060242.846154,2574825.211538,4139885.000000,4166199.000000
ID_FISC_LOG_DIFF,1839769.523810,1842175.066772,1833327.553460,1831038.572887,1821756.209447,1794303.195855,1822208.024505,1859189.451175,1765880.837903,1773367.700461,1652879.878261,2025158.270270,2222820.800000,2940649.000000,2877851.000000
I_FISC_LOGEMENT,0.996658,0.999978,0.999976,0.999956,0.999902,0.999802,0.999852,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Créer une dummy pour le foyer fiscal le plus riche !

In [25]:
df_ref_2 = df_ref[df_ref.nb_enf_log.isin([1,2,3,4,5,6,8,9,10,12])]

Repartition des enfants sur toutes les années 

In [26]:
#for nb_enf in range(2,12)[::1]:
    #print 'nb_enf:', nb_enf
    #print df_ref_2[df_ref_2.nb_enf_log == nb_enf].groupby('ID_FISC_LOG_DIFF')['nb_enf_foy'].max().value_counts()

In [27]:
df = pd.DataFrame(index = ["all on one", "equal split", "[0.6;0.7]", "[0.75]", "[0.8]"],
                  columns= pd.MultiIndex.from_product([range(2010,2015),
             ["{} child".format(nb_child) for nb_child in range(2,6)]]));

In [28]:
for year in range(2010,2015):
    df_ref_3 = df_ref_2[df_ref_2.AN_FISC == year+1]
    print year
    for nb_child in range(2,6)[::1]:

        #print 'nb_enf:', nb_enf
        #print df_ref_3[df_ref_3.nb_enf_log == nb_enf].groupby('ID_FISC_LOG_DIFF')['nb_enf_foy'].max().value_counts()

        df.loc["all on one", (year,"{} child".format(nb_child))
                  ] =  (df_ref_3[df_ref_3.nb_enf_log == nb_child].groupby('ID_FISC_LOG_DIFF')['nb_enf_foy'].max()==nb_child).sum()
        df.loc["equal split", (year,"{} child".format(nb_child))
          ] =  (df_ref_3[df_ref_3.nb_enf_log == nb_child].groupby('ID_FISC_LOG_DIFF')['nb_enf_foy'].max()==(nb_child/2)).sum()
        df.loc["[0.75]", (year,"{} child".format(nb_child))
          ] =  (df_ref_3[df_ref_3.nb_enf_log == nb_child].groupby('ID_FISC_LOG_DIFF')['nb_enf_foy'].max()/nb_child==0.75).sum()

        df.loc["[0.6;0.7]", (year,"{} child".format(nb_child))
            ] = ((df_ref_3[df_ref_3.nb_enf_log == nb_child].groupby('ID_FISC_LOG_DIFF')['nb_enf_foy'].max()/nb_child).isin([2/3,0.6])).sum()
        df.loc["[0.8]", (year,"{} child".format(nb_child))
            ] = ((df_ref_3[df_ref_3.nb_enf_log == nb_child].groupby('ID_FISC_LOG_DIFF')['nb_enf_foy'].max()/nb_child).isin([0.8])).sum()

        

        #print (df_ref_3[df_ref_3.nb_enf_log == nb_child].groupby('ID_FISC_LOG_DIFF')['nb_enf_foy'].max()/nb_child).value_counts()


df.index.name = 'Children share allocation'
df

2010
2011
2012
2013
2014


2010                            2011          \
                          2 child 3 child 4 child 5 child 2 child 3 child   
Children share allocation                                                   
all on one                  46098   12969    2640     648   47549   13388   
equal split                 24876       0    3069       0   26262       0   
[0.6;0.7]                       0    8599       0     782       0    9040   
[0.75]                          0       0    1473       0       0       0   
[0.8]                           0       0       0     300       0       0   

                                             2012                          \
                          4 child 5 child 2 child 3 child 4 child 5 child   
Children share allocation                                                   
all on one                   2699     700   48186   13653    2787     771   
equal split                  3430       0   26808       0    3665       0   
[0.6;0.7]                       0     875       0    9531       0     911   
[0.75]                       1629       0       0       0    1708       0   
[0.8]                           0     318       0       0       0     341   

                             2013                            2014          \
                          2 child 3 child 4 child 5 child 2 child 3 child   
Children share allocation                                                   
all on one                  48150   13791    2849     787   48397   13966   
equal split                 26670       0    3911       0   27105       0   
[0.6;0.7]                       0    9674       0     991       0   10019   
[0.75]                          0       0    1845       0       0       0   
[0.8]                           0       0       0     359       0       0   

                                           
                          4 child 5 child  
Children share allocation                  
all on one                   3010     826  
equal split                  4340       0  
[0.6;0.7]                       0    1112  
[0.75]                       1867       0  
[0.8]                           0     363

In [29]:
## En relatif

for year in range(2010,2015):
    df_ref_3 = df_ref_2[df_ref_2.AN_FISC == year+1]
    print year
    for nb_child in range(2,6)[::1]:

        #print 'nb_enf:', nb_enf
        #print df_ref_3[df_ref_3.nb_enf_log == nb_enf].groupby('ID_FISC_LOG_DIFF')['nb_enf_foy'].max().value_counts()

        df.loc["all on one", (year,"{} child".format(nb_child))
                  ] =  (df_ref_3[df_ref_3.nb_enf_log == nb_child].groupby('ID_FISC_LOG_DIFF')['nb_enf_foy'].max()==nb_child
                       ).sum()/df_ref_3[df_ref_3.nb_enf_log == nb_child].ID_FISC_LOG_DIFF.nunique()
        df.loc["equal split", (year,"{} child".format(nb_child))
                  ] =  (df_ref_3[df_ref_3.nb_enf_log == nb_child].groupby('ID_FISC_LOG_DIFF')['nb_enf_foy'].max()==(nb_child/2)
                       ).sum()/df_ref_3[df_ref_3.nb_enf_log == nb_child].ID_FISC_LOG_DIFF.nunique()
        df.loc["[0.75]", (year,"{} child".format(nb_child))
          ] =  (df_ref_3[df_ref_3.nb_enf_log == nb_child].groupby('ID_FISC_LOG_DIFF')['nb_enf_foy'].max()/nb_child==0.75).sum(
                        )/df_ref_3[df_ref_3.nb_enf_log == nb_child].ID_FISC_LOG_DIFF.nunique()

        df.loc["[0.6;0.7]", (year,"{} child".format(nb_child))
            ] = ((df_ref_3[df_ref_3.nb_enf_log == nb_child].groupby('ID_FISC_LOG_DIFF')['nb_enf_foy'].max()/nb_child).isin([2/3,0.6])
                ).sum()/df_ref_3[df_ref_3.nb_enf_log == nb_child].ID_FISC_LOG_DIFF.nunique()
        df.loc["[0.8]", (year,"{} child".format(nb_child))
            ] = ((df_ref_3[df_ref_3.nb_enf_log == nb_child].groupby('ID_FISC_LOG_DIFF')['nb_enf_foy'].max()/nb_child).isin([0.8])
                ).sum()/df_ref_3[df_ref_3.nb_enf_log == nb_child].ID_FISC_LOG_DIFF.nunique()

        

        #print (df_ref_3[df_ref_3.nb_enf_log == nb_child].groupby('ID_FISC_LOG_DIFF')['nb_enf_foy'].max()/nb_child).value_counts()


df.index.name = 'Children share allocation'
df

2010
2011
2012
2013
2014


2010                                   \
                             2 child    3 child    4 child    5 child   
Children share allocation                                               
all on one                 0.6495055  0.6013075  0.3675856  0.3745665   
equal split                0.3504945          0  0.4273183          0   
[0.6;0.7]                          0  0.3986925          0  0.4520231   
[0.75]                             0          0  0.2050961          0   
[0.8]                              0          0          0  0.1734104   

                                2011                                   \
                             2 child    3 child    4 child    5 child   
Children share allocation                                               
all on one                 0.6441994  0.5969324  0.3478989  0.3697834   
equal split                0.3558006          0  0.4421243          0   
[0.6;0.7]                          0  0.4030676          0  0.4622293   
[0.75]                             0          0  0.2099768          0   
[0.8]                              0          0          0  0.1679873   

                                2012                                   \
                             2 child    3 child    4 child    5 child   
Children share allocation                                               
all on one                 0.6425314  0.5888975  0.3415441  0.3811172   
equal split                0.3574686          0  0.4491422          0   
[0.6;0.7]                          0  0.4111025          0  0.4503213   
[0.75]                             0          0  0.2093137          0   
[0.8]                              0          0          0  0.1685615   

                                2013                                   \
                             2 child    3 child    4 child    5 child   
Children share allocation                                               
all on one                 0.6435445  0.5877264  0.3310866  0.3682733   
equal split                0.3564555          0  0.4545032          0   
[0.6;0.7]                          0  0.4122736          0  0.4637342   
[0.75]                             0          0  0.2144102          0   
[0.8]                              0          0          0  0.1679925   

                                2014                                   
                             2 child    3 child    4 child    5 child  
Children share allocation                                              
all on one                 0.6410029  0.5822806  0.3265705  0.3589744  
equal split                0.3589971          0   0.470869          0  
[0.6;0.7]                          0  0.4177194          0  0.4832681  
[0.75]                             0          0  0.2025605          0  
[0.8]                              0          0          0  0.1577575

In [30]:
print df.to_latex()

\begin{tabular}{lllllllllllllllllllll}
\toprule
{} &       2010 &            &            &            &       2011 &            &            &            &       2012 &            &            &            &       2013 &            &            &            &       2014 &            &            &            \\
{} &    2 child &    3 child &    4 child &    5 child &    2 child &    3 child &    4 child &    5 child &    2 child &    3 child &    4 child &    5 child &    2 child &    3 child &    4 child &    5 child &    2 child &    3 child &    4 child &    5 child \\
\midrule
Children share allocation &            &            &            &            &            &            &            &            &            &            &            &            &            &            &            &            &            &            &            &            \\
all on one                &  0.6495055 &  0.6013075 &  0.3675856 &  0.3745665 &  0.6441994 &  0.5969324 &  0.3478989 &  0.

In [31]:
(df_ref_3[df_ref_3.nb_enf_log == 2].groupby('ID_FISC_LOG_DIFF')['nb_enf_foy'].max()==2).sum()

48397

In [32]:
#df.loc["all on one", (2012,'1 child')] = 36

In [33]:
df

2010                                   \
                             2 child    3 child    4 child    5 child   
Children share allocation                                               
all on one                 0.6495055  0.6013075  0.3675856  0.3745665   
equal split                0.3504945          0  0.4273183          0   
[0.6;0.7]                          0  0.3986925          0  0.4520231   
[0.75]                             0          0  0.2050961          0   
[0.8]                              0          0          0  0.1734104   

                                2011                                   \
                             2 child    3 child    4 child    5 child   
Children share allocation                                               
all on one                 0.6441994  0.5969324  0.3478989  0.3697834   
equal split                0.3558006          0  0.4421243          0   
[0.6;0.7]                          0  0.4030676          0  0.4622293   
[0.75]                             0          0  0.2099768          0   
[0.8]                              0          0          0  0.1679873   

                                2012                                   \
                             2 child    3 child    4 child    5 child   
Children share allocation                                               
all on one                 0.6425314  0.5888975  0.3415441  0.3811172   
equal split                0.3574686          0  0.4491422          0   
[0.6;0.7]                          0  0.4111025          0  0.4503213   
[0.75]                             0          0  0.2093137          0   
[0.8]                              0          0          0  0.1685615   

                                2013                                   \
                             2 child    3 child    4 child    5 child   
Children share allocation                                               
all on one                 0.6435445  0.5877264  0.3310866  0.3682733   
equal split                0.3564555          0  0.4545032          0   
[0.6;0.7]                          0  0.4122736          0  0.4637342   
[0.75]                             0          0  0.2144102          0   
[0.8]                              0          0          0  0.1679925   

                                2014                                   
                             2 child    3 child    4 child    5 child  
Children share allocation                                              
all on one                 0.6410029  0.5822806  0.3265705  0.3589744  
equal split                0.3589971          0   0.470869          0  
[0.6;0.7]                          0  0.4177194          0  0.4832681  
[0.75]                             0          0  0.2025605          0  
[0.8]                              0          0          0  0.1577575

### Allocation by year biologic

In [34]:
for year in range(2010,2015):
    biologic_path = (u"./Pickle/biologic/biologic_households{}.p".format(year))
    biologic = pickle.load( open(biologic_path, 'rb'))
    
    df_ref_3 = df_ref_2[df_ref_2.AN_FISC == year+1]
    df_ref_3_biologic = df_ref_3[df_ref_3.ID_FISC_LOG_DIFF.isin(biologic)]
    print year
    for nb_child in range(2,6)[::1]:

        #print 'nb_enf:', nb_enf
        #print df_ref_3_biologic[df_ref_3_biologic.nb_enf_log == nb_enf].groupby('ID_FISC_LOG_DIFF')['nb_enf_foy'].max().value_counts()

        df.loc["all on one", (year,"{} child".format(nb_child))
                  ] =  (df_ref_3_biologic[df_ref_3_biologic.nb_enf_log == nb_child].groupby('ID_FISC_LOG_DIFF')['nb_enf_foy'].max()==nb_child).sum()
        df.loc["equal split", (year,"{} child".format(nb_child))
          ] =  (df_ref_3_biologic[df_ref_3_biologic.nb_enf_log == nb_child].groupby('ID_FISC_LOG_DIFF')['nb_enf_foy'].max()==(nb_child/2)).sum()
        df.loc["[0.75]", (year,"{} child".format(nb_child))
          ] =  (df_ref_3_biologic[df_ref_3_biologic.nb_enf_log == nb_child].groupby('ID_FISC_LOG_DIFF')['nb_enf_foy'].max()/nb_child==0.75).sum()

        df.loc["[0.6;0.7]", (year,"{} child".format(nb_child))
            ] = ((df_ref_3_biologic[df_ref_3_biologic.nb_enf_log == nb_child].groupby('ID_FISC_LOG_DIFF')['nb_enf_foy'].max()/nb_child).isin([2/3,0.6])).sum()
        df.loc["[0.8]", (year,"{} child".format(nb_child))
            ] = ((df_ref_3_biologic[df_ref_3_biologic.nb_enf_log == nb_child].groupby('ID_FISC_LOG_DIFF')['nb_enf_foy'].max()/nb_child).isin([0.8])).sum()

        

        #print (df_ref_3_biologic[df_ref_3_biologic.nb_enf_log == nb_child].groupby('ID_FISC_LOG_DIFF')['nb_enf_foy'].max()/nb_child).value_counts()



df.index.name = 'Children share allocation'
df

2010
2011
2012
2013
2014


2010                            2011          \
                          2 child 3 child 4 child 5 child 2 child 3 child   
Children share allocation                                                   
all on one                   9645    1059     109       8   11049    1300   
equal split                  6178       0     484       0    7137       0   
[0.6;0.7]                       0     423       0       9       0     549   
[0.75]                          0       0      18       0       0       0   
[0.8]                           0       0       0       3       0       0   

                                             2012                          \
                          4 child 5 child 2 child 3 child 4 child 5 child   
Children share allocation                                                   
all on one                    126      16   12126    1577     150      26   
equal split                   579       0    7729       0     668       0   
[0.6;0.7]                       0      11       0     646       0      12   
[0.75]                         27       0       0       0      31       0   
[0.8]                           0       1       0       0       0       0   

                             2013                            2014          \
                          2 child 3 child 4 child 5 child 2 child 3 child   
Children share allocation                                                   
all on one                  12906    1773     168      30   13643    2011   
equal split                  8111       0     764       0    8664       0   
[0.6;0.7]                       0     726       0      15       0     791   
[0.75]                          0       0      37       0       0       0   
[0.8]                           0       0       0       3       0       0   

                                           
                          4 child 5 child  
Children share allocation                  
all on one                    218      31  
equal split                   874       0  
[0.6;0.7]                       0      19  
[0.75]                         41       0  
[0.8]                           0       0

In [35]:
## En relatif

for year in range(2010,2015):
    biologic_path = (u"./Pickle/biologic/biologic_households{}.p".format(year))
    biologic = pickle.load( open(biologic_path, 'rb'))
    
    df_ref_3 = df_ref_2[df_ref_2.AN_FISC == year+1]
    df_ref_3_biologic = df_ref_3[df_ref_3.ID_FISC_LOG_DIFF.isin(biologic)]
    print year
    for nb_child in range(2,6)[::1]:

        #print 'nb_enf:', nb_enf
        #print df_ref_3_biologic[df_ref_3_biologic.nb_enf_log == nb_enf].groupby('ID_FISC_LOG_DIFF')['nb_enf_foy'].max().value_counts()

        df.loc["all on one", (year,"{} child".format(nb_child))
                  ] =  (df_ref_3_biologic[df_ref_3_biologic.nb_enf_log == nb_child].groupby('ID_FISC_LOG_DIFF')['nb_enf_foy'].max()==nb_child
                       ).sum()/df_ref_3_biologic[df_ref_3_biologic.nb_enf_log == nb_child].ID_FISC_LOG_DIFF.nunique()
        df.loc["equal split", (year,"{} child".format(nb_child))
                  ] =  (df_ref_3_biologic[df_ref_3_biologic.nb_enf_log == nb_child].groupby('ID_FISC_LOG_DIFF')['nb_enf_foy'].max()==(nb_child/2)
                       ).sum()/df_ref_3_biologic[df_ref_3_biologic.nb_enf_log == nb_child].ID_FISC_LOG_DIFF.nunique()
        df.loc["[0.75]", (year,"{} child".format(nb_child))
          ] =  (df_ref_3_biologic[df_ref_3_biologic.nb_enf_log == nb_child].groupby('ID_FISC_LOG_DIFF')['nb_enf_foy'].max()/nb_child==0.75).sum(
                        )/df_ref_3_biologic[df_ref_3_biologic.nb_enf_log == nb_child].ID_FISC_LOG_DIFF.nunique()

        df.loc["[0.6;0.7]", (year,"{} child".format(nb_child))
            ] = ((df_ref_3_biologic[df_ref_3_biologic.nb_enf_log == nb_child].groupby('ID_FISC_LOG_DIFF')['nb_enf_foy'].max()/nb_child).isin([2/3,0.6])
                ).sum()/df_ref_3_biologic[df_ref_3_biologic.nb_enf_log == nb_child].ID_FISC_LOG_DIFF.nunique()
        df.loc["[0.8]", (year,"{} child".format(nb_child))
            ] = ((df_ref_3_biologic[df_ref_3_biologic.nb_enf_log == nb_child].groupby('ID_FISC_LOG_DIFF')['nb_enf_foy'].max()/nb_child).isin([0.8])
                ).sum()/df_ref_3_biologic[df_ref_3_biologic.nb_enf_log == nb_child].ID_FISC_LOG_DIFF.nunique()

        

        #print (df_ref_3[df_ref_3.nb_enf_log == nb_child].groupby('ID_FISC_LOG_DIFF')['nb_enf_foy'].max()/nb_child).value_counts()


df.index.name = 'Children share allocation'
df

2010
2011
2012
2013
2014


2010                                     2011  \
                             2 child    3 child    4 child 5 child    2 child   
Children share allocation                                                       
all on one                 0.6095557  0.7145749  0.1783961     0.4  0.6075553   
equal split                0.3904443          0   0.792144       0  0.3924447   
[0.6;0.7]                          0  0.2854251          0    0.45          0   
[0.75]                             0          0  0.0294599       0          0   
[0.8]                              0          0          0    0.15          0   

                                                                   2012  \
                             3 child     4 child     5 child    2 child   
Children share allocation                                                 
all on one                 0.7030827   0.1721311   0.5714286  0.6107278   
equal split                        0   0.7909836           0  0.3892722   
[0.6;0.7]                  0.2969173           0   0.3928571          0   
[0.75]                             0  0.03688525           0          0   
[0.8]                              0           0  0.03571429          0   

                                                                  2013  \
                             3 child     4 child    5 child    2 child   
Children share allocation                                                
all on one                 0.7094017   0.1766784  0.6842105  0.6140743   
equal split                        0    0.786808          0  0.3859257   
[0.6;0.7]                  0.2905983           0  0.3157895          0   
[0.75]                             0  0.03651355          0          0   
[0.8]                              0           0          0          0   

                                                               2014  \
                             3 child     4 child 5 child    2 child   
Children share allocation                                             
all on one                 0.7094838   0.1733746   0.625  0.6116017   
equal split                        0   0.7884417       0  0.3883983   
[0.6;0.7]                  0.2905162           0  0.3125          0   
[0.75]                             0  0.03818369       0          0   
[0.8]                              0           0  0.0625          0   

                                                          
                             3 child     4 child 5 child  
Children share allocation                                 
all on one                 0.7177016   0.1924095    0.62  
equal split                        0   0.7714034       0  
[0.6;0.7]                  0.2822984           0    0.38  
[0.75]                             0  0.03618711       0  
[0.8]                              0           0       0

In [36]:
stop_time = datetime.datetime.now(); print("now: ",stop_time);
execution_time = stop_time - start_time; print("execution_time: ",execution_time)
stop_time = start_time
#clear  memory
%reset -f

('now: ', datetime.datetime(2019, 4, 26, 18, 37, 17, 903000))
('execution_time: ', datetime.timedelta(0, 476, 145000))
